In [2]:
import pandas as pd
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from mlflow.models import infer_signature

"""
1. 데이터 로딩
2. 고객 이탈 예측을 위한 데이터
3. 'Exited' 컬럼이 예측 대상(target)
"""
data = pd.read_csv('../data/churn.csv')

# Tracking Server 설정
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") # 환경 변수를 설정했지만, 확실하게 설정

In [3]:
# 불필요한 컬럼 제거 후 feature(X), target(y) 분리
X = data.drop(['Exited', 'RowNumber', 'CustomerId', 'Surname'], axis=1)
y = data['Exited']

In [4]:
# 범주형 수치형 feature 정의
categorical_features = ['Geography', 'Gender']
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']

In [5]:
# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features), # 수치형 데이터 표준화
        ('cat', OneHotEncoder(), categorical_features) # 범주형 데이터 원핫인코딩
    ])

In [6]:
# 데이터 전처리 수행
X_processed = preprocessor.fit_transform(X)

In [7]:
# 데이터셋 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [8]:
# MLFlow 실험 설정
mlflow.set_experiment("practice2")

2025/10/10 11:48:47 INFO mlflow.tracking.fluent: Experiment with name 'practice2' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///C:/Users/SSAFY/Desktop/TIL/100_offline/8. DA/82일차 '
 '(10.10)/data_science2_ws_7_2/mlflow_project/mlruns/1'), creation_time=1760064527205, experiment_id='1', last_update_time=1760064527205, lifecycle_stage='active', name='practice2', tags={}>

In [9]:
# XGBoost 모델 학습 및 평가
with mlflow.start_run(run_name="XGBoost"):
    model = xgb.XGBClassifier(n_estimators=100, eval_metric='logloss')
    model.fit(X_train, y_train) # 모델 학습
    preds = model.predict(X_test) # 예측 수행

    # 성능 평가 지표 계산
    acc = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)

    # MLflow에 하이퍼파라미터 및 평가 지표 저장
    mlflow.log_params({'model': 'XGBoost', 'n_estimators': 100})
    mlflow.log_metrics({
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    })

    # 입력과 출력 형태 정의 및 MLFlow에 모델 저장
    signature = infer_signature(X_train, preds)
    
    # 'registered_model_name' 파라미터를 지정하면 모델이 자동으로 Model Registry에 등록
    mlflow.xgboost.log_model(model, "xgboost_model", signature=signature, model_format='json', registered_model_name="Practice2_ChurnModel")

Successfully registered model 'Practice2_ChurnModel'.
2025/10/10 11:49:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Practice2_ChurnModel, version 1


🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/1/runs/aae44ecfd4ad4bd387d23fadf8fd7709
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Created version '1' of model 'Practice2_ChurnModel'.


In [10]:
from mlflow.tracking import MlflowClient

# MLflow 클라이언트 객체 생성
client = MlflowClient()

# 모델 버전 1에 'staging'이라는 alias 설정
# 기존 stage='Staging'과 같은 의미를 alias='staging'으로 표현
client.set_registered_model_alias(
    name="Practice2_ChurnModel",  # 모델 이름
    alias="staging",              # 설정할 alias (예: staging, production 등)
    version=1                     # alias가 가리킬 모델 버전
)


In [11]:
# 이 코드는 주석처리 후, 이전까지의 코드가 모두 수행된 상태

# 등록된 모델 불러오기 및 예측 수행
model = mlflow.pyfunc.load_model("models:/Practice2_ChurnModel@staging")
predictions = model.predict(X_test)
print(predictions)

[0 0 0 ... 1 0 1]
